# Hello distributed world

The goal of this tutorial is to setup a simulation and run it by using minimal example.

In [1]:
# If pydistsim is installed (e.g via pip install pydistsim)
from pydistsim import *

# If pydistsim is not installed and the script is run from docs/notebooks directory
# import sys
# sys.path.append("../../")
# from pydistsim import *

set_log_level("WARNING")

# Create a network with 100 nodes
net_gen = NetworkGenerator(100)
net = net_gen.generate_random_network()

# Assign a simple algorithm to the network
from pydistsim.algorithms.broadcast import Flood

net.algorithms = ((Flood, {"informationKey": "I"}),)
some_node = tuple(net.nodes())[0]
some_node.memory["I"] = "Hello distributed world"

# Create and run the simulation
sim = Simulation(net)
sim.run()

That's it! For more elaborate description please continue reading.

In [2]:
# For more detailed output, set the log level to INFO
set_log_level("INFO")

In the example given above, the goal was to simulate distributed algorithm `Flood` on arbitrary network. Algorithm `Flood` solves *Broadcast* problem, i.e. the goal of the algorithm is to share information that is present in one or more nodes to all other nodes in the network.

In this example the IPython interactive console is started with `ipydistsim` or `ipython --profile=pydistsim`. This way certain names from PyDistSim package should already be available such as `Node` or `Network`.

### Generating network

Networks are fundamental objects in the PyDistSim and they can be created by instantiating `Network` class or by using  `NetworkGenerator` class. `NetworkGenerator` can receive different parameters such as number of nodes (exact, min, max), average number of neighbors per node etc.

In this example the only parametar that we want to have in control is number of nodes. which is set to 100.

In [3]:
net_gen = NetworkGenerator(100)

Now generator's methods can be used to generate new networks with specified parameters, i.e. for generating network with random placement of nodes there is method `generate_random_network()`:

In [4]:
net = net_gen.generate_random_network()

*Note:* One of the `NetworkGenerator` parameters control if the generated network should be connected (which means  there are no isolated segments of the network). This parameter `connected` is by default set to `True` so if the number of nodes is set too low then some generator's methods  might not be able to produce the network.

The result is instance of the Network class:

In [5]:
net

which can be visualized with its `show()` method:

/home/agus/.pymote_venv/lib/python3.11/site-packages/networkx/drawing/nx_pylab.py:450: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  node_collection = ax.scatter(
/mnt/d/Proyectos/pymote/docs/notebooks/../../pydistsim/network.py:340: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  fig.show()


In [6]:
net.show()

### Algorithm

Algorithm `Flood` is part of the `broadcast` module so it should be imported:

In [7]:
from pydistsim.algorithms.broadcast import Flood

and then assigned to the generated network:

In [8]:
net.algorithms = ((Flood, {"informationKey": "I"}),)

Two things should be noted:

* there can be multiple algorithms that are being assigned to the network so specific algorithms are elements of `tuple`. Since in this example there is only one algorithm we must append `,` so that Python knows it's a one element `tuple` i.e. `(1)` is `int`, but `(1,)` is `tuple`.
* every algorithm element is `tuple` itself, consisting of two elements. Former is an algorithm class, in this example `Flood`. Later is a `dict` of keyword parameters, i.e. `Flood` must be given 'informationKey' as a required parameter: that is the name of the key under which the nodes should store broadcasted information in their memory.


This algorithm demands that at least one node knows information to be broadcasted, so in this example at least one node should have some value stored under `I` key:

In [9]:
some_node = list(net.nodes())[0]  # take the first node, it can be any node
some_node.memory["I"] = (
    "Hello distributed world"  # under key 'I' write the information to be boradcasted
)

 This way `some_node` has become algorithm initiator.

### Simulation

After network creation and setup all that is left to do is to create and run simulation. Every simulation has its own network (sent as parameter) that it runs on:

In [10]:
sim = Simulation(net)

Finally, run simulation:

In [11]:
sim.run()

After the algorithm(s) execution is done we can check if the resullt is as expected, that is every node should have information in their memory:

In [12]:
for node in net.nodes():
    assert node.memory["I"] == "Hello distributed world"
print("Everything is OK!")

Everything is OK!


During simulation execution, network and its nodes are changing their status, memory content etc. To memorize the currently running algorithm and current step of the simulation, network has a special attribute `algorithmsState`:

In [13]:
net.algorithmState

{'index': 0, 'step': 27, 'finished': True}

If simulation is going to be run again, it must be reset:

In [14]:
sim.reset()

whereby:

* `algorithmState` is returned to the initial value:

In [15]:
net.algorithmState

{'index': 0, 'step': 1, 'finished': False}

* memory content of all node's is deleted (as long as some other attributes)

In [16]:
for node in net.nodes():
    assert len(node.memory) == 0
print("All memories are empty!")

All memories are empty!


In [17]:
some_node.memory["I"] = "Hello again"
sim.run()